In [1]:
import os
import openai
import chromadb
from chromadb import Settings
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter  # LangChain의 텍스트 스플리터
from whoosh.index import create_in, open_dir
from whoosh.fields import Schema, TEXT, ID
from whoosh.qparser import QueryParser
import fitz  # PyMuPDF
import pandas as pd
import uuid  # For generating unique IDs

# Set up OpenAI API key using environment variables
os.environ["OPENAI_API_KEY"] = "yourapikey"

# Initialize OpenAI Embeddings
openai_embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# Step 1: Data Preprocessing - PDF and CSV Handling with LangChain's RecursiveCharacterTextSplitter
def chunk_text(text, chunk_size=300, chunk_overlap=50):
    """LangChain의 RecursiveCharacterTextSplitter로 텍스트를 분할합니다."""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    return text_splitter.split_text(text)

def extract_text_from_pdf(pdf_path):
    """PDF에서 텍스트를 추출하고 분할하여 반환합니다."""
    doc = fitz.open(pdf_path)
    text_data = []
    for page_num in range(doc.page_count):
        page = doc[page_num]
        text = page.get_text("text")
        chunks = chunk_text(text)  # LangChain의 텍스트 분할 사용
        for chunk_num, chunk in enumerate(chunks, 1):
            text_data.append({
                "content": chunk,
                "page_num": page_num + 1,
                "chunk_num": chunk_num,
                "source": pdf_path
            })
    return text_data

def extract_data_from_csv(csv_path):
    """CSV에서 텍스트를 추출하여 반환합니다."""
    print(f"Reading CSV file: {csv_path}")  # 파일 경로 출력
    df = pd.read_csv(csv_path)
    text_data = []
    for i, row in df.iterrows():
        row_text = row.to_string()
        text_data.append({
            "content": row_text,
            "row_num": i + 1,
            "source": csv_path
        })
    return text_data

# Step 2: Initialize Chromadb with persistent storage
chroma_client = chromadb.PersistentClient(
    path="./chroma1105",
    settings=Settings(allow_reset=True)
)
collection = chroma_client.get_or_create_collection("document_collection")

# Step 3: Initialize Whoosh for BM25 with persistent storage
schema = Schema(content=TEXT(stored=True), source=ID(stored=True), page_or_row_num=ID(stored=True), chunk_num=ID(stored=True))
index_dir = "indexdir"
if os.path.exists(index_dir):
    import shutil
    shutil.rmtree(index_dir)
os.mkdir(index_dir)
ix = create_in(index_dir, schema)

def index_data(data):
    """데이터를 Chroma와 Whoosh 인덱스에 추가합니다."""
    with ix.writer() as writer:
        for entry in data:
            unique_id = str(uuid.uuid4())
            
            # Embedding for Chromadb
            embedding = openai_embeddings.embed_query(entry["content"])
            metadata = {
                "source": entry["source"], 
                "page_or_row_num": entry["page_num" if "page_num" in entry else "row_num"]
            }
            
            # CSV 데이터의 경우 chunk_num을 생략하거나 기본값 설정
            if "chunk_num" in entry:
                metadata["chunk_num"] = entry["chunk_num"]
            
            collection.add(
                ids=[unique_id],
                embeddings=[embedding],
                documents=[entry["content"]],
                metadatas=[metadata]
            )
            
            # Index for BM25 (Whoosh)
            writer.add_document(
                content=entry["content"],
                source=entry["source"],
                page_or_row_num=str(entry["page_num" if "page_num" in entry else "row_num"]),
                chunk_num=str(entry.get("chunk_num", 1))  # chunk_num이 없으면 기본값 1로 설정
            )
            print(f"Indexed document: {entry['content'][:30]}...")  # Debug: Show part of the indexed document


# Step 4: Process all PDF and CSV files in folders
def process_folder(pdf_folder, csv_folder):
    all_data = []
    
    for filename in os.listdir(pdf_folder):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder, filename)
            pdf_data = extract_text_from_pdf(pdf_path)
            all_data.extend(pdf_data)
    
    for filename in os.listdir(csv_folder):
        if filename.endswith(".csv"):
            csv_path = os.path.join(csv_folder, filename)
            csv_data = extract_data_from_csv(csv_path)
            all_data.extend(csv_data)
    
    index_data(all_data)

# Specify folders
pdf_folder = "./pdf1"
csv_folder = "./pdf3CSV"
process_folder(pdf_folder, csv_folder)

# Step 5: Search Functionality
def search(query, top_k=5):
    # Semantic Search (Chromadb)
    query_embedding = openai_embeddings.embed_query(query)
    chromadb_results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )

    # Keyword Search (BM25 with Whoosh)
    keyword_results = []
    with ix.searcher() as searcher:
        single_word_query = query.split()[0]
        query_parser = QueryParser("content", ix.schema)
        whoosh_query = query_parser.parse(single_word_query)
        print(f"Whoosh query: {whoosh_query}")

        whoosh_results = searcher.search(whoosh_query, limit=top_k)
        for hit in whoosh_results:
            keyword_results.append({
                "content": hit["content"],
                "source": hit["source"],
                "page_or_row_num": hit["page_or_row_num"],
                "chunk_num": hit["chunk_num"],
                "score": hit.score
            })
            print(f"Found document: {hit['content'][:30]}... with score {hit.score}")

    results = {
        "semantic_results": chromadb_results['documents'],
        "keyword_results": keyword_results
    }
    return results

# Sample query search
query = "Example query text"
results = search(query)
print("Semantic Results:", results["semantic_results"])
print("Keyword Results:", results["keyword_results"])


/home/daun/miniconda3/envs/rag/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


ParserError: Error tokenizing data. C error: Expected 10 fields in line 6, saw 12


In [1]:
import os
import openai
import faiss  # FAISS 가져오기
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from whoosh.index import create_in, open_dir
from whoosh.fields import Schema, TEXT, ID
from whoosh.qparser import QueryParser
import fitz  # PyMuPDF
import pandas as pd
import numpy as np
import pickle  # 메타데이터 저장용
import uuid  # 고유 ID 생성을 위해

# OpenAI API 키 설정
os.environ["OPENAI_API_KEY"] = "yourkey"

# OpenAI Embeddings 초기화
openai_embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# FAISS 인덱스 초기화
embedding_dim = 1536  # `text-embedding-ada-002` 모델의 차원
index = faiss.IndexFlatL2(embedding_dim)
faiss_ids = []
metadata_store = {}

# 텍스트를 분할하는 함수 정의
def chunk_text(text, chunk_size=300, chunk_overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return text_splitter.split_text(text)

# PDF에서 텍스트 추출 후 분할
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text_data = []
    for page_num in range(doc.page_count):
        page = doc[page_num]
        text = page.get_text("text")
        chunks = chunk_text(text)
        for chunk_num, chunk in enumerate(chunks, 1):
            text_data.append({
                "content": chunk,
                "page_num": page_num + 1,
                "chunk_num": chunk_num,
                "source": pdf_path
            })
    return text_data

# CSV 파일에서 데이터 추출
def extract_data_from_csv(csv_path):
    df = pd.read_csv(csv_path)
    text_data = []
    for i, row in df.iterrows():
        row_text = row.to_string()
        text_data.append({
            "content": row_text,
            "row_num": i + 1,
            "source": csv_path
        })
    return text_data

# Whoosh로 BM25 인덱스 초기화
schema = Schema(content=TEXT(stored=True), source=ID(stored=True), page_or_row_num=ID(stored=True), chunk_num=ID(stored=True))
index_dir = "indexdir1107"
if os.path.exists(index_dir):
    import shutil
    shutil.rmtree(index_dir)
os.mkdir(index_dir)
ix = create_in(index_dir, schema)

# 데이터 인덱싱 함수
def index_data(data):
    with ix.writer() as writer:
        for entry in data:
            unique_id = str(uuid.uuid4())
            
            # FAISS용 임베딩
            embedding = openai_embeddings.embed_query(entry["content"])
            embedding_np = np.array([embedding], dtype="float32")
            
            # 메타데이터에 'content' 추가
            metadata = {
                "content": entry["content"],  # content 추가
                "source": entry["source"], 
                "page_or_row_num": entry["page_num" if "page_num" in entry else "row_num"],
                "chunk_num": entry.get("chunk_num", 1)
            }
            
            faiss_ids.append(unique_id)
            metadata_store[unique_id] = metadata
            index.add(embedding_np)  # FAISS 인덱스에 추가

            # Whoosh BM25 인덱싱
            writer.add_document(
                content=entry["content"],
                source=entry["source"],
                page_or_row_num=str(entry["page_num" if "page_num" in entry else "row_num"]),
                chunk_num=str(entry.get("chunk_num", 1))
            )

# 폴더 내 PDF 및 CSV 파일 처리
def process_folder(pdf_folder, csv_folder):
    all_data = []
    for filename in os.listdir(pdf_folder):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder, filename)
            pdf_data = extract_text_from_pdf(pdf_path)
            all_data.extend(pdf_data)
    
    for filename in os.listdir(csv_folder):
        if filename.endswith(".csv"):
            csv_path = os.path.join(csv_folder, filename)
            csv_data = extract_data_from_csv(csv_path)
            all_data.extend(csv_data)
    
    index_data(all_data)

# FAISS 인덱스와 메타데이터 저장 함수
def save_faiss_index(index, faiss_ids, metadata_store, index_path="faiss_index1107.bin", metadata_path="metadata_store1107.pkl"):
    faiss.write_index(index, index_path)
    print("FAISS 인덱스가 저장되었습니다.")
    
    # 메타데이터 저장
    with open(metadata_path, "wb") as f:
        pickle.dump((faiss_ids, metadata_store), f)
    print("메타데이터가 저장되었습니다.")

# FAISS 인덱스와 메타데이터 불러오기 함수
def load_faiss_index(index_path="faiss_index1107.bin", metadata_path="metadata_store1107.pkl"):
    index = faiss.read_index(index_path)
    print("FAISS 인덱스를 성공적으로 불러왔습니다.")
    
    # 메타데이터 불러오기
    with open(metadata_path, "rb") as f:
        faiss_ids, metadata_store = pickle.load(f)
    print("메타데이터를 성공적으로 불러왔습니다.")
    
    return index, faiss_ids, metadata_store

# 폴더 경로 설정 및 인덱싱 수행 후 저장
pdf_folder = "./pdf1"
csv_folder = "./pdf3CSV"
process_folder(pdf_folder, csv_folder)
save_faiss_index(index, faiss_ids, metadata_store)

# 검색 함수
def search(query, top_k=5):
    # 저장된 인덱스와 메타데이터 불러오기
    index, faiss_ids, metadata_store = load_faiss_index()
    
    # FAISS 유사도 검색
    query_embedding = np.array([openai_embeddings.embed_query(query)], dtype="float32")
    _, faiss_indices = index.search(query_embedding, top_k)
    
    semantic_results = []
    for idx in faiss_indices[0]:
        if idx < len(faiss_ids):
            unique_id = faiss_ids[idx]
            metadata = metadata_store[unique_id]
            semantic_results.append({
                "content": metadata["content"],
                "source": metadata["source"],
                "page_or_row_num": metadata["page_or_row_num"],
                "chunk_num": metadata["chunk_num"]
            })

    # Whoosh로 키워드 검색
    keyword_results = []
    with ix.searcher() as searcher:
        query_parser = QueryParser("content", ix.schema)
        whoosh_query = query_parser.parse(query)
        whoosh_results = searcher.search(whoosh_query, limit=top_k)
        for hit in whoosh_results:
            keyword_results.append({
                "content": hit["content"],
                "source": hit["source"],
                "page_or_row_num": hit["page_or_row_num"],
                "chunk_num": hit["chunk_num"],
                "score": hit.score
            })

    return {
        "semantic_results": semantic_results,
        "keyword_results": keyword_results
    }

# 샘플 검색
query = "Example query text"
results = search(query)
print("Semantic Results:", results["semantic_results"])
print("Keyword Results:", results["keyword_results"])


/home/daun/miniconda3/envs/rag/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


FAISS 인덱스가 저장되었습니다.
메타데이터가 저장되었습니다.
FAISS 인덱스를 성공적으로 불러왔습니다.
메타데이터를 성공적으로 불러왔습니다.
Semantic Results: [{'content': '외야 펜스가 상대적으로 낮거나 가까워 홈런이 나기 쉬운 구조를 가지고 있습니다. 이는 \n타자들이 더 쉽게 점수를 낼 수 있는 환경을 제공합니다."} \n \n{"inputs": "선수들이 경기 할 때는 개인 경기도구를 사용해도 돼?", "response": "KBO 리\n그의 경우, 개인 장비를 사용할 수 있지만 대회 사무국이 공인한 제품만 사용이 가능합\n니다."} \n{"inputs": "보크가 뭐야?", "response": "주자가 있을 때 투수가 저지르는 변칙적인 투구동\n작입니다."}', 'source': './pdf1/야구 지식.pdf', 'page_or_row_num': 3, 'chunk_num': 2}, {'content': '기의 변수를 창출해 내는 능력이 필요합니다."} \n \n \n{"inputs": "클린업 트리오는 어떤 역할을 해?", "response": "3, 4, 5번 타자를 일컫는 말로, \n보통 팀 내 최고의 선수들을 이 타선에 배치하며 야구에서 득점할 때 가장 중요한 역할\n을 하는 타선입니다. 미국에서는 4번 타자만을 얘기하지만 일본에서 3, 4, 5를 한꺼번에 \n일컬으면서 4번 타자가 그 중에서도 팀의 상징이라는 것으로 변질됐습니다."}', 'source': './pdf1/야구 지식.pdf', 'page_or_row_num': 6, 'chunk_num': 1}, {'content': '작입니다."} \n{"inputs": "인필드플라이가 뭐야?", "response": "무사나 1사 상황에서 1, 2루 혹은 만루 상\n황에 놓여있을 때, 타자가 내야수가 정상적인 수비로 잡을 수 있는 페어 플라이 볼을 치\n는 것으로, 심판이 이것을 선언하면 수비수가 공을 잡는 것과 관계없이 타자는 아웃이 \n됩니다."} \n{"in

In [2]:
# Set up OpenAI API key using environment variables
# os.environ["OPENAI_API_KEY"] = "yourkey"

# pdf_folder = "./pdf1"
# csv_folder = "./pdf3CSV"

In [12]:
# Sample query search
query = "롯데 야구 잘하는 것 같애 못 하는 것 같애"
results = search(query)
print("Semantic Results:", results["semantic_results"])
print("Keyword Results:", results["keyword_results"])


FAISS 인덱스를 성공적으로 불러왔습니다.
메타데이터를 성공적으로 불러왔습니다.
Semantic Results: [{'content': '면 한번은 뛰어야 하는 팀’으로 인식될 정도였다.\n여담으로 자매 팀과는 정반대다. 응원만큼은 롯데 자이언츠만큼이나 열정적이나, 일본에서 \n가장 인기 없는 야구팀 중 하나로 꼽힌다. 다만 성적만큼은 치바 쪽이 좋은 편이다. 특히 한국 \n롯데에겐 아직도 먼 21세기 우승을 치바는 두 번이나 해냈다(2005, 2010). 더욱이 2005년\n도 우승은 당시 상대 팀에게 씻을 수 없는 모욕감을 선사했다.\n부산경남 지역 민영방송인 KNN 라디오에서는 롯데 자이언츠 전 경기 생중계를 한다. 수요', 'source': './pdf1/롯데.pdf', 'page_or_row_num': 7, 'chunk_num': 3}, {'content': '경기 많은 홈경기를 치뤘으며, 홈 최종전까지 3위 싸움을 하는 박진감 넘치는 순위 싸움을 했\n음에도, 최종적으로 홈 96만 관중 기록에 그치며 5년 만의 100만 관중 달성에 실패했다.\n이는 야구를 처음 접한 사람들이 단순히 두산의 성적이 좋다는 이유로 응원을 하다가, 성적\n이 떨어지면 야구에 대한 관심을 끊는 등 팬심이 상대적으로 끈끈하지 않다는 것을 증명한\n다. 그럼에도, 2014년 새로 부임한 응원단장 한재권이 엄청난 능력을 보여주면서 무려 외야', 'source': './pdf1/두산.pdf', 'page_or_row_num': 6, 'chunk_num': 4}, {'content': '진영의 빈 공간으로 떨어뜨리는 쪽이 점수를 획득한다. 그러나 야구에서는 투수가 포수에게 \n던지는 공을 상대팀 타자가 방망이로 쳐내야 하며, 아무리 공을 잘, 많이 쳐내도 주자가 홈 베\n이스를 밟지 못하면 점수가 나지 않으며, 이런 식으로 아웃 카운트 세 개가 모두 잡힐 때까지 \n홈에 들어오지 못한 주자를 잔루라 한다.\n즉, 다른 구기가 공을 다루는 기술에 역점을 두어 발전해 왔다